# 创建阶段：执行上下文与提升


[ES6 规范中的词法环境与闭包、作用域的关系](./lexical_env.ipynb) 里经常说词法环境是在代码执行时创建的。

但是代码「执行时」到底是什么时候？

接下来我们将拆解代码执行过程，进一步了解代码执行细节。

JS 代码的执行实际上分为两个阶段：创建阶段和执行阶段。

回顾下 [简述 V8 引擎对 JS 代码的处理流程](./src/dive_into_javascript/notebooks/v8_pipeline.ipynb)。

JS 代码如果在加载之后就被执行，那么代码会被解析成完整的 AST 抽象语法树并由 V8 解释器转化成字节码执行，否则代码仅被预解析，预解析仅生成简化的 AST。

因此如果一段代码没有被解析过，需要在创建阶段之前先对代码进行真正的解析。

在创建阶段，V8 会创建一个执行上下文作为代码的执行环境。

执行上下文有两个重要的状态组件：

- 变量环境（Variable Environment）
- 词法环境（Lexical Environment）

注意，执行上下文中的词法环境和 ES6 规范中的词法环境不是一回事。

ES6 规范中的词法环境（Lexical Environment）在 [ES6 规范中的词法环境与闭包、作用域的关系](./lexical_env.ipynb) 里已经介绍过了，
它是一个包含环境记录和对外词法环境引用的结构，是结构。

而执行上下文中的词法环境（LexicalEnvironment）和变量环境是词法环境（Lexical Environment）结构的状态组件，是组件。

并且，词法环境组件和变量环境组件是词法环境结构的。

_词法环境（Lexical Environment）和词法环境（LexicalEnvironment）这两个英文单词，一个有空格，一个没有，有空格的是结构，没空格的是组件_

#### 提升

这里插入一个概念：**「提升」**，理解「提升」有利于深入理解执行上下文。

JS 里「提升」的意思是变量和函数在实际声明之前就已经存在于其所属作用域。

所有的变量声明和函数声明都会被提升。

当 JS 执行全局/模块/函数代码的时候（代码执行的创建阶段）会创建对应的执行上下文，执行上下文初始化阶段会创建变量环境和词法环境组件（引用？），
在当前执行上下文中，块级作用域之外的 var 变量和函数声明会被添加到执行上下文的变量环境组件的环境记录中，`const` 和 `let` 变量被添加到词法环境组件的环境记录。 var 被初始化为 undefined，函数声明以完整的函数定义的形式被记录，const 和 let 不会被初始化，在实际声明之前不能被访问，这就是暂时性死区。

#### 块级作用域的实现

[ES6 规范对代码块的描述](https://262.ecma-international.org/6.0/#sec-block-runtime-semantics-evaluation)

[How is block scope managed in the lexical environment?](https://stackoverflow.com/questions/29410249/how-is-block-scope-managed-in-the-lexical-environment)

如果代码块不是空代码块。

1. 保存当前词法环境组件的引用 oldEnv
2. 创建一个新的词法环境组件，引用是 oldEnv，意味着新环境继承了旧环境的标识符绑定，但它自己也可以有新的绑定
3. 将当前执行上下文的词法环境设置为 blockEnv。这意味着从现在起，块内的所有变量和函数声明都会绑定到 blockEnv。
4. 代码块执行完之后，词法环境组件恢复成 oldEnv

一般的作用域都是通过执行上下文中的词法环境组件实现的，但是代码块没有执行上下文。
虽然没有块级词法环境但是通过这种机制，实现了块级作用域。

词法环境组件是变化的，变量环境组件是不变的。

[What is the temporal dead zone?](https://stackoverflow.com/questions/33198849/what-is-the-temporal-dead-zone)

[[AskJS] how is hoisting significant in javascript?](https://www.reddit.com/r/javascript/comments/u5yf8m/askjs_how_is_hoisting_significant_in_javascript/)


### 作用域、词法环境、执行上下文三者关系
